In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from apyori import apriori
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [28]:
# read data
file = 'game_teams_stats.csv'

#read CSV into df
data = pd.read_csv(file)

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52610 entries, 0 to 52609
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_id                 52610 non-null  int64  
 1   team_id                 52610 non-null  int64  
 2   HoA                     52610 non-null  object 
 3   won                     52610 non-null  bool   
 4   settled_in              52610 non-null  object 
 5   head_coach              52582 non-null  object 
 6   goals                   52602 non-null  float64
 7   shots                   52602 non-null  float64
 8   hits                    47682 non-null  float64
 9   pim                     52602 non-null  float64
 10  powerPlayOpportunities  52602 non-null  float64
 11  powerPlayGoals          52602 non-null  float64
 12  faceOffWinPercentage    30462 non-null  float64
 13  giveaways               47682 non-null  float64
 14  takeaways               47682 non-null

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,blocked,startRinkSide
0,2016020045,4,away,False,REG,Dave Hakstol,4.0,27.0,30.0,6.0,4.0,2.0,50.9,12.0,9.0,11.0,left
1,2016020045,16,home,True,REG,Joel Quenneville,7.0,28.0,20.0,8.0,3.0,2.0,49.1,16.0,8.0,9.0,left
2,2017020812,24,away,True,OT,Randy Carlyle,4.0,34.0,16.0,6.0,3.0,1.0,43.8,7.0,4.0,14.0,right
3,2017020812,7,home,False,OT,Phil Housley,3.0,33.0,17.0,8.0,2.0,1.0,56.2,5.0,6.0,14.0,right
4,2015020314,21,away,True,REG,Patrick Roy,4.0,29.0,17.0,9.0,3.0,1.0,45.7,13.0,5.0,20.0,left


In [29]:
#data = data.dropna()    #dropping rows with NA-values reduced entries from 52610 to 29554

#create a column for pp%
data['powerPlayPercentage'] = (data['powerPlayGoals'] / data['powerPlayOpportunities'])

#reorder columns
column_order = ['game_id', 'team_id', 'HoA', 'won', 'settled_in', 'head_coach', 'goals', 'shots', 'hits', 'pim', 'powerPlayOpportunities', 'powerPlayGoals', 'powerPlayPercentage', 'faceOffWinPercentage', 'giveaways', 'takeaways', 'blocked', 'startRinkSide']
data = data[column_order]
data = data.groupby('game_id').filter(lambda x: len(x) == 2)
data.info()



#make an aleternative df where numerical values are normalized to [0, 1]
scaler = MinMaxScaler()
num_cols = data.select_dtypes(include=['float64', 'int64']).columns
data_norm = data.copy()
data_norm[num_cols] = scaler.fit_transform(data[num_cols])
data_norm.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42330 entries, 0 to 46669
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_id                 42330 non-null  int64  
 1   team_id                 42330 non-null  int64  
 2   HoA                     42330 non-null  object 
 3   won                     42330 non-null  bool   
 4   settled_in              42330 non-null  object 
 5   head_coach              42302 non-null  object 
 6   goals                   42322 non-null  float64
 7   shots                   42322 non-null  float64
 8   hits                    37402 non-null  float64
 9   pim                     42322 non-null  float64
 10  powerPlayOpportunities  42322 non-null  float64
 11  powerPlayGoals          42322 non-null  float64
 12  powerPlayPercentage     41806 non-null  float64
 13  faceOffWinPercentage    20182 non-null  float64
 14  giveaways               37402 non-null

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,powerPlayPercentage,faceOffWinPercentage,giveaways,takeaways,blocked,startRinkSide
0,0.841661,0.056604,away,False,REG,Dave Hakstol,0.333333,0.421875,0.3750,0.028169,0.2500,0.285714,0.500000,0.642677,0.230769,0.225,0.255814,left
1,0.841661,0.283019,home,True,REG,Joel Quenneville,0.583333,0.437500,0.2500,0.037559,0.1875,0.285714,0.666667,0.619949,0.307692,0.200,0.209302,left
2,0.894305,0.433962,away,True,OT,Randy Carlyle,0.333333,0.531250,0.2000,0.028169,0.1875,0.142857,0.333333,0.553030,0.134615,0.100,0.325581,right
3,0.894305,0.113208,home,False,OT,Phil Housley,0.250000,0.515625,0.2125,0.037559,0.1250,0.142857,0.500000,0.709596,0.096154,0.150,0.325581,right
4,0.789071,0.377358,away,True,REG,Patrick Roy,0.333333,0.453125,0.2125,0.042254,0.1875,0.142857,0.333333,0.577020,0.250000,0.125,0.465116,left


In [30]:
home_team_df = data[data['HoA'] == 'home'].set_index('game_id')
away_team_df = data[data['HoA'] == 'away'].set_index('game_id')

# Merge the two DataFrames based on 'game_id'
combined_df = home_team_df.merge(away_team_df, on='game_id', suffixes=('_home', '_away'))

# Reset the index to have 'game_id' as a column
combined_df.reset_index(inplace=True)
combined_df = combined_df.reindex(sorted(combined_df.columns), axis=1)
new_order = ['game_id'] + [col for col in combined_df.columns if col != 'game_id']
combined_df = combined_df[new_order]
combined_df = combined_df.drop(['faceOffWinPercentage_away', 'faceOffWinPercentage_home', 'HoA_away', 'HoA_home', 'head_coach_away', 'head_coach_home', 'settled_in_away', 'startRinkSide_away', 'startRinkSide_home'], axis=1)
combined_df.rename(columns={'settled_in_home': 'settled_in'}, inplace=True)

print(combined_df.iloc[5])
combined_df = combined_df[combined_df["goals_home"] != combined_df["goals_away"]]
combined_df.info()
combined_df.head()

game_id                        2016020610
blocked_away                         24.0
blocked_home                         18.0
giveaways_away                        8.0
giveaways_home                       12.0
goals_away                            4.0
goals_home                            1.0
hits_away                            19.0
hits_home                            27.0
pim_away                              8.0
pim_home                              4.0
powerPlayGoals_away                   1.0
powerPlayGoals_home                   1.0
powerPlayOpportunities_away           2.0
powerPlayOpportunities_home           4.0
powerPlayPercentage_away              0.5
powerPlayPercentage_home             0.25
settled_in                            REG
shots_away                           39.0
shots_home                           23.0
takeaways_away                        5.0
takeaways_home                        7.0
team_id_away                           15
team_id_home                      

,game_id,blocked_away,blocked_home,giveaways_away,giveaways_home,goals_away,goals_home,hits_away,hits_home,pim_away,...,powerPlayPercentage_home,settled_in,shots_away,shots_home,takeaways_away,takeaways_home,team_id_away,team_id_home,won_away,won_home
0,2016020045,11.0,9.0,12.0,16.0,4.0,7.0,30.0,20.0,6.0,...,0.666667,REG,27.0,28.0,9.0,8.0,4,16,False,True
1,2017020812,14.0,14.0,7.0,5.0,4.0,3.0,16.0,17.0,6.0,...,0.500000,OT,34.0,33.0,4.0,6.0,24,7,True,False
2,2015020314,20.0,9.0,13.0,13.0,4.0,1.0,17.0,22.0,9.0,...,0.000000,REG,29.0,21.0,5.0,7.0,21,52,True,False
3,2015020849,16.0,13.0,4.0,12.0,1.0,2.0,21.0,16.0,10.0,...,0.400000,REG,21.0,29.0,14.0,11.0,52,12,False,True
4,2017020586,7.0,21.0,10.0,13.0,1.0,2.0,20.0,15.0,19.0,...,0.166667,REG,23.0,41.0,4.0,4.0,20,24,False,True


## DROP DATA THAT GIVES AWAY THE RESULT; GOALS ETC

In [31]:
updated_df = combined_df.drop(["game_id", "goals_away", "goals_home"], axis=1)
updated_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18635 entries, 0 to 21164
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   blocked_away                 16472 non-null  float64
 1   blocked_home                 16472 non-null  float64
 2   giveaways_away               16472 non-null  float64
 3   giveaways_home               16472 non-null  float64
 4   hits_away                    16472 non-null  float64
 5   hits_home                    16472 non-null  float64
 6   pim_away                     18631 non-null  float64
 7   pim_home                     18631 non-null  float64
 8   powerPlayGoals_away          18631 non-null  float64
 9   powerPlayGoals_home          18631 non-null  float64
 10  powerPlayOpportunities_away  18631 non-null  float64
 11  powerPlayOpportunities_home  18631 non-null  float64
 12  powerPlayPercentage_away     18349 non-null  float64
 13  powerPlayPercent

## Make data binary for association rule mining

In [49]:
# Assuming you have a DataFrame named 'df'

# Create a new DataFrame with relevant columns
new_df = combined_df[["won_home", "won_away", "shots_home", "shots_away", "blocked_home", "blocked_away", "takeaways_home", "takeaways_away", "hits_home", "hits_away", "pim_home", "pim_away", "powerPlayGoals_home", "powerPlayGoals_away", "powerPlayOpportunities_home", "powerPlayOpportunities_away", "powerPlayPercentage_home", "powerPlayPercentage_away"]]

# Define a function to determine the winner for each category (1 for home, 0 for away)
def determine_winner(row, home_col, away_col):
    if row[home_col] > row[away_col]:
        return 1
    else:
        return 0

# Apply the function to calculate winners for each category
new_df["win"] = new_df.apply(lambda row: 1 if row["won_home"] else 0, axis=1)
new_df["shots_winner"] = new_df.apply(lambda row: determine_winner(row, "shots_home", "shots_away"), axis=1)
new_df["blocks_winner"] = new_df.apply(lambda row: determine_winner(row, "blocked_home", "blocked_away"), axis=1)
new_df["takeaways_winner"] = new_df.apply(lambda row: determine_winner(row, "takeaways_home", "takeaways_away"), axis=1)
new_df["hits_winner"] = new_df.apply(lambda row: determine_winner(row, "hits_home", "hits_away"), axis=1)
#new_df["pim_winner"] = new_df.apply(lambda row: 1 if row["pim_home"] < row["pim_away"] else 0, axis=1)
#new_df["powerPlayGoals_winner"] = new_df.apply(lambda row: determine_winner(row, "powerPlayGoals_home", "powerPlayGoals_away"), axis=1)
new_df["powerPlayOpportunities_winner"] = new_df.apply(lambda row: determine_winner(row, "powerPlayOpportunities_home", "powerPlayOpportunities_away"), axis=1)
#new_df["powerPlayPercentage_winner"] = new_df.apply(lambda row: determine_winner(row, "powerPlayPercentage_home", "powerPlayPercentage_away"), axis=1)

# Drop the unnecessary columns
new_df = new_df.drop(["won_home", "won_away", "shots_home", "shots_away", "blocked_home", "blocked_away", "takeaways_home", "takeaways_away", "hits_home", "hits_away", "pim_home", "pim_away", "powerPlayGoals_home", "powerPlayGoals_away", "powerPlayOpportunities_home", "powerPlayOpportunities_away", "powerPlayPercentage_home", "powerPlayPercentage_away"], axis=1)

# Show the resulting DataFrame
new_df.info()


C:\Users\macka\AppData\Local\Temp/ipykernel_15424/1843748422.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["win"] = new_df.apply(lambda row: 1 if row["won_home"] else 0, axis=1)
C:\Users\macka\AppData\Local\Temp/ipykernel_15424/1843748422.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["shots_winner"] = new_df.apply(lambda row: determine_winner(row, "shots_home", "shots_away"), axis=1)
C:\Users\macka\AppData\Local\Temp/ipykernel_15424/1843748422.py:16: SettingWithCopyWarning: 
A va

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18635 entries, 0 to 21164
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   win                            18635 non-null  int64
 1   shots_winner                   18635 non-null  int64
 2   blocks_winner                  18635 non-null  int64
 3   takeaways_winner               18635 non-null  int64
 4   hits_winner                    18635 non-null  int64
 5   powerPlayOpportunities_winner  18635 non-null  int64
dtypes: int64(6)
memory usage: 1019.1 KB


C:\Users\macka\AppData\Local\Temp/ipykernel_15424/1843748422.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["powerPlayOpportunities_winner"] = new_df.apply(lambda row: determine_winner(row, "powerPlayOpportunities_home", "powerPlayOpportunities_away"), axis=1)


In [48]:
# Apply the Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(new_df, min_support=0.1, use_colnames=True)

# Find association rules
association_rules_df = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort on a column
association_rules_df = association_rules_df.sort_values(by="confidence", ascending=False)

# Display the association rules
print(association_rules_df)
association_rules_df.head()

                           antecedents  \
35   (takeaways_winner, blocks_winner)   
47         (pim_winner, blocks_winner)   
19       (blocks_winner, shots_winner)   
36                (win, blocks_winner)   
3                      (blocks_winner)   
..                                 ...   
48                               (win)   
99                  (takeaways_winner)   
100                              (win)   
101                     (shots_winner)   
84                  (takeaways_winner)   

                                      consequents  antecedent support  \
35                                          (win)            0.245237   
47                                          (win)            0.151918   
19                                          (win)            0.164368   
36                             (takeaways_winner)            0.255648   
3                                           (win)            0.369949   
..                                            ...        

c:\Users\macka\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
35,"(takeaways_winner, blocks_winner)",(win),0.245237,0.557338,0.181325,0.739387,1.326640,0.044645,1.698542,0.326216
47,"(pim_winner, blocks_winner)",(win),0.151918,0.557338,0.111779,0.735782,1.320172,0.027109,1.675368,0.285966
19,"(blocks_winner, shots_winner)",(win),0.164368,0.557338,0.119506,0.727065,1.304531,0.027898,1.621857,0.279358
36,"(win, blocks_winner)",(takeaways_winner),0.255648,0.535176,0.181325,0.709278,1.325318,0.044509,1.598861,0.329769
3,(blocks_winner),(win),0.369949,0.557338,0.255648,0.691036,1.239885,0.049461,1.432727,0.307077


## Try to classify with the "best" rules

In [52]:
# Define the features and target variable
X = new_df[["blocks_winner", "takeaways_winner", "hits_winner", "shots_winner"]]
y = new_df["win"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

Accuracy: 0.6138985779447277
Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.50      0.54      1671
           1       0.63      0.71      0.67      2056

    accuracy                           0.61      3727
   macro avg       0.61      0.60      0.60      3727
weighted avg       0.61      0.61      0.61      3727

